# AUB - MSFEA  
# EECE798S - Agentic Systems  
## Assignment 3: EnerSense — The Smart Energy Business Agent  
### *Prepared by Hadi Al Mubasher*  
#### *hma154@mail.aub.edu*  

---

EnerSense is a professional AI-powered business assistant that represents a fictitious company, **EnerSense Analytics** — a consultancy specializing in energy monitoring, automated reporting, and sustainability analytics.  
The agent can answer questions about the company, collect customer leads, schedule consultations, generate report and monitoring requests, and log customer feedback.  

Built using **OpenAI GPT-4o-mini** and deployed through a **Gradio interface**, EnerSense integrates intelligent tool-calling to execute structured backend functions such as:  
- `record_customer_interest()` for leads  
- `schedule_consultation()` for bookings  
- `list_available_slots()` for availability checks  
- `log_site_monitoring_request()` and `log_energy_report_request()` for service setup  

All logs are automatically saved under `/logs/`, ensuring realistic business traceability.  
The project demonstrates prompt-driven reasoning, modular design, and seamless integration between natural language understanding and structured tool invocation.


In [1]:
from tools import (
    record_customer_interest,
    record_feedback,
    log_site_monitoring_request,
    log_energy_report_request,
    schedule_consultation
)

import json
from typing import List, Dict

In [2]:
import pdfplumber
import os

def load_business_docs():
    """
    Loads and concatenates the EnerSense business profile files
    (summary text + PDF content) into a single reference string.

    Returns:
    --------
    - str: Combined business description used for system context.
    """
    base_path = "me"
    summary_path = os.path.join(base_path, "business_summary.txt")
    pdf_path = os.path.join(base_path, "about_business.pdf")

    # Read text summary
    with open(summary_path, "r", encoding="utf-8") as f:
        summary_text = f.read().strip()

    # Extract text from the PDF using pdfplumber
    pdf_text = ""
    with pdfplumber.open(pdf_path) as pdf:
        for page in pdf.pages:
            pdf_text += page.extract_text() + "\n"

    # Combine both
    combined_text = f"{summary_text}\n\n{pdf_text}"
    return combined_text.strip()

business_docs = load_business_docs()

In [3]:
system_prompt = f"""
You are EnerSense — the virtual assistant of EnerSense Analytics,
an energy consultancy specializing in smart monitoring, automated reporting,
and sustainability analytics for industrial and commercial clients.

Use the following company information to answer questions naturally:
{business_docs}

Your personality and goals:
- Represent EnerSense as professional, knowledgeable, and helpful.
- Communicate clearly and concisely, like a technical consultant.
- Understand the user’s intent and respond intelligently — ask for missing details only when essential.
- When appropriate, use your available tools to record leads, feedback, consultations, reports, or monitoring requests.

Guidelines for tool use (interpret flexibly, not mechanically):
- Use **record_customer_interest** when the user expresses interest or inquiry in EnerSense’s services (especially with name/email).
- Use **log_site_monitoring_request** only if the user clearly provides a specific site or facility name (e.g., “MTC North”, “EcoGrid Factory”)
  along with monitoring parameters. Do not ask for a site name if it isn’t given.
- Use **log_energy_report_request** when the user explicitly asks for an energy report with a company name and period.
- Use **schedule_consultation** when the user provides name, date, and topic, or **list_available_slots** if they ask for available times.
- Use **record_feedback** for off-topic questions or anything unrelated to EnerSense’s services.

Behavior:
- Be natural. Infer intent from context rather than matching exact keywords.
- If multiple interpretations are possible, choose the one that best fits a helpful EnerSense representative.
- Avoid re-confirming obvious information — acknowledge, act, and move forward confidently.
- When no tool is needed, respond conversationally using company knowledge.

Stay consistently in character as EnerSense — a courteous, intelligent, and reliable energy consultant.
"""


In [4]:
tools: List[Dict] = [
    {
        "type": "function",
        "function": {
            "name": "record_customer_interest",
            "description": (
                "Record potential customer interest as soon as name and email are provided. "
                "Use the user's request text as the 'message'. "
                "This function is used for general inquiries when no specific site or monitoring setup is mentioned."
            ),
            "parameters": {
                "type": "object",
                "properties": {
                    "name": {
                        "type": "string",
                        "description": "Full name of the interested customer."
                    },
                    "email": {
                        "type": "string",
                        "description": "Customer's email address."
                    },
                    "message": {
                        "type": "string",
                        "description": "Inquiry or type of service the customer is interested in."
                    }
                },
                "required": ["name", "email", "message"]
            }
        }
    },
    {
        "type": "function",
        "function": {
            "name": "record_feedback",
            "description": (
                "Log a question, comment, or feedback that the assistant cannot confidently answer. "
                "Used for collecting off-topic or unclear inquiries for review."
            ),
            "parameters": {
                "type": "object",
                "properties": {
                    "question": {
                        "type": "string",
                        "description": "User's question or feedback text."
                    }
                },
                "required": ["question"]
            }
        }
    },
    {
        "type": "function",
        "function": {
            "name": "log_site_monitoring_request",
            "description": (
                "Log a monitoring setup request only when the user explicitly provides a distinct, named site or facility "
                "(e.g., 'MTC North', 'Plant A', 'EcoGrid Factory'). "
                "If the user uses generic phrases such as 'my facility', 'our site', or 'the building', "
                "do NOT call this function — instead, treat the request as general customer interest "
                "and call record_customer_interest(name, email, message)."
            ),
            "parameters": {
                "type": "object",
                "properties": {
                    "customer_name": {
                        "type": "string",
                        "description": "Name of the customer requesting the monitoring setup."
                    },
                    "site_name": {
                        "type": "string",
                        "description": (
                            "Distinct and explicit name of the site or facility (e.g., 'Plant A', 'EcoGrid Factory'). "
                            "Generic terms like 'my facility' are not valid here."
                        )
                    },
                    "parameters": {
                        "type": "string",
                        "description": "Parameters or sensors to monitor, e.g., temperature, voltage, current."
                    }
                },
                "required": ["customer_name", "site_name", "parameters"]
            }
        }
    },
    {
        "type": "function",
        "function": {
            "name": "log_energy_report_request",
            "description": (
                "Log a customer's request for an energy performance report. "
                "The function requires a company name and a reporting period (e.g., weekly, monthly)."
            ),
            "parameters": {
                "type": "object",
                "properties": {
                    "company_name": {
                        "type": "string",
                        "description": "Name of the company requesting the report."
                    },
                    "period": {
                        "type": "string",
                        "description": "Desired reporting frequency (e.g., 'weekly', 'monthly', 'quarterly')."
                    }
                },
                "required": ["company_name", "period"]
            }
        }
    },
    {
        "type": "function",
        "function": {
            "name": "schedule_consultation",
            "description": (
                "Book a consultation for the client if the requested time slot is available. "
                "Used after confirming name, date, and topic details."
            ),
            "parameters": {
                "type": "object",
                "properties": {
                    "name": {
                        "type": "string",
                        "description": "Client's full name."
                    },
                    "date": {
                        "type": "string",
                        "description": "Requested consultation date and time (e.g., '2025-10-22 11:00')."
                    },
                    "topic": {
                        "type": "string",
                        "description": "Main topic or focus of the consultation."
                    }
                },
                "required": ["name", "date", "topic"]
            }
        }
    },
    {
        "type": "function",
        "function": {
            "name": "list_available_slots",
            "description": (
                "List available consultation time slots for a given date. "
                "If all slots are booked, the model should politely inform the user."
            ),
            "parameters": {
                "type": "object",
                "properties": {
                    "date": {
                        "type": "string",
                        "description": "Date in 'YYYY-MM-DD' format for which available consultation times are requested."
                    }
                },
                "required": ["date"]
            }
        }
    }
]

In [5]:
from openai import OpenAI
from dotenv import load_dotenv
import gradio as gr
import json

# Load .env file
load_dotenv()

# Read the API key from environment variable
client = OpenAI(api_key=os.getenv("OPENAI_API_KEY"))

# Import your tool functions
from tools import (
    record_customer_interest,
    record_feedback,
    log_site_monitoring_request,
    log_energy_report_request,
    schedule_consultation,
    list_available_slots
)

In [6]:
def call_tool(tool_name: str, args: dict):
    """
    Executes the appropriate tool function based on the tool call name.

    Inputs:
    -------
    - tool_name: str. Name of the called tool (from model output).
    - args: dict. Tool arguments parsed from the model’s tool call.

    Returns:
    --------
    - str: Tool output message for the chat response.
    """
    if tool_name == "record_customer_interest":
        return record_customer_interest(**args)
    elif tool_name == "record_feedback":
        return record_feedback(**args)
    elif tool_name == "log_site_monitoring_request":
        return log_site_monitoring_request(**args)
    elif tool_name == "log_energy_report_request":
        return log_energy_report_request(**args)
    elif tool_name == "schedule_consultation":
        return schedule_consultation(**args)
    elif tool_name == "list_available_slots":
        return list_available_slots(**args)
    else:
        return "Sorry, I couldn't process that request."


def chat_with_agent(user_input: str, history: list):
    """
    Handles the full chatbot conversation loop with multi-tool support.

    Inputs:
    -------
    - user_input: str. User message from the Gradio interface.
    - history: list. Chat history as [(user, assistant), ...].

    Returns:
    --------
    - str: Model-generated response text for display.
    """
    # Build message list with system prompt
    messages = [{"role": "system", "content": system_prompt}]

    # Reconstruct conversation history
    for user_msg, assistant_msg in history:
        messages.append({"role": "user", "content": user_msg})
        messages.append({"role": "assistant", "content": assistant_msg})

    # Add new user message
    messages.append({"role": "user", "content": user_input})

    # Request model completion with tool definitions
    response = client.chat.completions.create(
        model="gpt-4o-mini",
        messages=messages,
        tools=tools,
        tool_choice="auto"
    )

    message = response.choices[0].message

    # If the assistant called one or more tools
    if message.tool_calls:
        tool_messages = []  # list to store each tool result

        for tool_call in message.tool_calls:
            tool_name = tool_call.function.name
            tool_args = json.loads(tool_call.function.arguments)

            # Execute tool and capture result
            tool_result = call_tool(tool_name, tool_args)

            # Append tool result as a tool message (with proper tool_call_id)
            tool_messages.append({
                "role": "tool",
                "tool_call_id": tool_call.id,
                "content": tool_result
            })

        # Send all tool results back to the model for a natural follow-up
        follow_up = client.chat.completions.create(
            model="gpt-4o-mini",
            messages=messages + [message] + tool_messages
        )

        return follow_up.choices[0].message.content

    # No tool calls → return direct answer
    return message.content

In [7]:
def launch_interface():
    """
    Launches a Gradio ChatInterface for the EnerSense Virtual Assistant.
    """
    ui = gr.ChatInterface(
        fn=chat_with_agent,
        title="EnerSense Virtual Assistant",
        description="Ask about energy monitoring, reports, or consultations.",
        examples=[
            "Tell me about your services.",
            "I want to schedule a consultation next week.",
            "Can you prepare a monthly report for EcoGrid Ltd?",
            "I'd like to monitor temperature and current at our MTC North site."
        ],
        theme="default",
    )
    ui.launch()


# Run the chatbot
if __name__ == "__main__":
    launch_interface()


Running on local URL:  http://127.0.0.1:7860

To create a public link, set `share=True` in `launch()`.
